In [ ]:
import requests 
import requests.auth
import json
#from PIL import Image
#from io import BytesIO
from IPython.display import display, Image


# Digital Trace Data

The Internet is a gigantic data dump. There is all the social networking data from Facebook, Twitter, and so on. There is the news from all the traditional media sources plus Quartz, Vox, and so on. Then there is the data from organizations such as the World Bank, the Bureau of Labor Statistics, the US Census, or Chicago's Data Portal.  Finally, you have all your scientific data sources: the National Cancer Institute, the ProteinBank, or the Kyoto Gene and Genomes Encyclopedia.

How can you use Python to access those sites and retrieve data for your research, your business, or your hobby?

There are two main approaches to retrieve data from websources. The preferred approach is using **Application Program Interfaces** or APIs.  If an organization has decided to share its data, and they have the forethought and resources to do it, they will develop an API that will let you interact with their data.

If the organization does not have the forethought or resources to create an API (or if they do not want to share their data), then you have to **crawl** their website and **scrape** their data.


+++

# Application Program Interfaces (APIs)


APIs simplify the process of obtaining specific information from a data source.  You do not have to worry about figuring out the **format** in which the information is stored, or **where** the information is stored.  All of those matter are handled seamlessly by the API. 

But convenience is not the only advantage of an API. APIs are also particular useful when:

* **The data is a small part of a whole.** Reddit comments are one example. What if you want to just pull your own comments on Reddit? It doesn’t make much sense to download the entire Reddit database, then filter just your own comments.
    
* **Massive imbalance in data availability.** Spotify has an API that can tell you the genre of a piece of music. You could theoretically create your own classifier, and use it to categorize music, but you’ll never have as much data as Spotify does.
    
* **Data is changing quickly.** An example of this is stock price data. It doesn’t really make sense to regenerate a dataset and download it every minute – this will take a lot of bandwidth, and be pretty slow.


## Schematic of an API transaction

Using an API is essentially the same as viewing a web page at a url. The rough transaction is as follows:

<img src='../../images/api_schema.jpeg'>

1. Formulate and send the query to the server
2. The server runs the specified query on its database
3. Server returns the query data object

The difference between this process and a web page is that you are (i) not making a specific demand when you go to a web page and (ii) the server would return the data embedded in html script (a web page is data after all). 

## Returned data types

APIs will almost always return data in one of two formats: XML or JSON. These formats are nothing particularly special, they simply specify how the data object will be structured. XML is used **far less** than JSON, but both are presented here for completeness.

### eXtensible Markup Language (XML)

XML is a markup language, with the main difference being that it can accept arbitrary classes for items that would not be recognized in HTML. As an example:

<img src='../../images/xml_example.png'>

Each ''<>'' is called a tag, it's an opening tag when it has the name normally spelled (``<food>``) and a closing tag when it has the forward slash in front of the name (``</food>``). In order for an XML file to be read without issue, it is necessary that every open tag is closed. 

As far as the structure of the rest of the file (i.e. what line goes where), that is left to the author to decide. One navigates an XML file by moving from sibling to sibling (tags that are at the same level and next to each other) or parent to child (a tag that is inside of another tag).

### Javascript Object Notation (JSON)

JSON is a set of key-value pairs, where the keys must be unique. If this sounds familiar, it is because it is effectively the exact same definition for a dictionary in Python. When you look at a JSON, you will effectively see a dictionary.

<img src='../../images/json_example.png'>

The ``{}`` denotes the start and end of the data object and the key value pair is separated by the ``:``. Something to note - a python dictionary is not actually a JSON, you must convert it from the variable to the object.

In [ ]:
sample = {'key1': 'value1', 'key2': 'value2'}
sample

In [ ]:
import json 
json.dumps(sample)

You can tell that you have a JSON when it is represented as a string.

## Making requests

In order to learn how APIs work, we will first use the APIs developed to retrieve data on the **International Space Station (ISS)**.  The relevant APIs can be found at http://open-notify.org/.  We will first consider the API for retrieving the location (latitude and longitude) of the ISS (http://open-notify.org/Open-Notify-API/ISS-Location-Now/). The API is hosted at http://api.open-notify.org/iss-now.json. 

So, how do we make requests for information with this API?

Like standard webpages, APIs are also hosted on web servers. When you type http://www.google.com in your browser’s address bar, your computer is actually asking the http://www.google.com server for a webpage, which it then returns it to your browser for display. That action is called a `request`. 

There are many possible types of requests. The most common, and the one we will be using throughout this unit, is the `GET` request. A `GET` request simply accesses and downloads the webpage found at the URL you specified as an input. 

We will use the package [`requests`](http://docs.python-requests.org/en/latest/user/quickstart/) package to crawl (load) webpages and scrape (download) their contents.

In [ ]:
import requests 

response = requests.get("http://api.open-notify.org/iss-now.json")
print( response )
print( response.status_code )


Methods from the `requests` package return `Response` objects. One of the most important properties of the response is its `status code`, which is printed by default but which we can also get explicitly.

Here are some of the most common status codes you might encounter:
* 200, **OK**. Standard response for successful HTTP requests. The actual response will depend on the request method used.
* 301, **Moved Permanently**. The server is redirecting you to a different endpoint. This and all future requests should be directed to the given URL. This can happen when a company switches domain names, or an endpoint name is changed.
* 303, **See Other**. The response to the request can be found under another URI using a GET method. When received in response to a POST (or PUT/DELETE), the client should presume that the server has received the data and should issue a redirect with a separate GET message. Your web browser automatically fetches the new URL but web crawlers do not usually do this unless you specify it.
* 400, **Bad Request**. The server cannot or will not process the request due to an apparent client error (e.g., malformed request syntax, invalid request message framing, or deceptive request routing).
* 401, **Unauthorized**. Similar to `403 Forbidden`, but specifically for use when authentication is required and has failed or has not yet been provided. The response must include a WWW-Authenticate header field containing a challenge applicable to the requested resource.
* 403, **Forbidden**. The request was a valid request, but the server is refusing to respond to it. `403` error semantically means "unauthorized", i.e. the user does not have the necessary permissions for the resource.
* 404, **Not Found**. The requested resource could not be found but may be available in the future. Subsequent requests by the client are permissible.
* 500, **Internal Server Error**. A generic error message, given when an `unexpected` condition was encountered and no more specific message is suitable.
* 503, **Service Unavailable**. The server is currently unavailable (because it is overloaded or down for maintenance). Generally, this is a temporary state.
* 504, **Gateway Timeout**. The server was acting as a gateway or proxy and did not receive a timely response from the upstream server.[



More codes: http://en.wikipedia.org/wiki/List_of_HTTP_status_codes

The status code of our request was **200**. It means that all went well -- we successfully connected to the web address we wanted and downloaded its contents.

But `status codes` are not the only methods available:

In [ ]:
print( response.url )

In [ ]:
response.text

This is the content format specified http://open-notify.org/Open-Notify-API/ISS-Location-Now/. It is in `json` format which means that we can easily parse it using the `json` module.

In [ ]:
data = json.loads(response.text)

data

**YES**. 

The method `loads()` returns json formatted data as a dictionary. We can print whatever information we need from the dictionary using the appropriate `keys`.

In [ ]:
print( f"""The ISS current position is {data['iss_position']['latitude']} of latitude and {data['iss_position']['longitude']} of longitude.""")

**Exercise:** It is now time for you to try to use an API on your own. The last API available at [`Open Notify`](http://open-notify.org/) returns the number of astronauts in the ISS. Write the code to access that information.

## Using The US Census' API

The United States Census is a decennial census mandated by the United States Constitution. The United States Census Bureau (officially the Bureau of the Census) is responsible for the United States Census.

The first census after the American Revolution was taken in 1790, under Secretary of State Thomas Jefferson; there have been 22 federal censuses since that time. The current national census was held in 2010; the next census is scheduled for 2020 and will be largely conducted using the Internet. For years between the decennial censuses, the Census Bureau issues estimates made using surveys and statistical models.

The Census Bureau has begun rolling out their datasets via [APIs](http://www.census.gov/developers/). You can find a full list of APIs [here](http://www.census.gov/data/developers/data-sets.html).  In this unit, we will focus on the decennial census.

Because we are dealing with US data, we will start by loading some helpful data: US city names, their states, and their geographic codes.  The relevant data is stored in `json` format in `../data/`



In [ ]:
ls ../../data/us*json

In [ ]:
import json
with open('../../data/us_state_names.json') as file_in:
    states_w_codes = json.load( file_in )  
    
states_w_codes

In [ ]:
with open('../../data/us_state_city_names.json') as file_in:
    cities_by_state = json.load( file_in )
    
cities_by_state

**FIPS state codes** are numeric and two-letter alphabetic codes defined in U.S. Federal Information Processing Standard Publication ("FIPS PUB") 5-2 to identify U.S. states and certain other associated areas. The codes are used in Geographic Names Information System, overseen by the U.S. Board on Geographic Names. 

In [ ]:
print(states_w_codes['MT'])
print(cities_by_state['MT']['Antelope'])

Now that we have the basic information, we can start using the API to retrieve data. The Census Bureau has a number of helpful resources.  The decennial census page constains basic instructions on how to contruct queries. There is a also a page with example, and a page with a list of all (and I *really* mean **all**) variable codes.

But before we can do anything, we need to obtain a `key` that will identify us as the person doing the queries.

This is an important first step in learning how to interact with an API. 

**Exercise.** Request and obtain an API key to use with the Census. 

In [ ]:
!cp /Users/adampah/Dropbox/Teaching/NICO-101/2019Materials/Lessons/census_auth.json ../../data/

In [ ]:
auth = json.load(open('../../data/census_auth.json'))
auth

Now I would like you to retrieve the total population for all of the states from the following url http://api.census.gov/data/2010/sf1

In [ ]:
import requests
census_url = 'https://api.census.gov/data/2010/dec/sf1'
response = requests.get( census_url, params = {'key': auth['my_key'], 
                                               'get': 'P001001,NAME', 
                                               'for': 'state: *'})
print(response.status_code)
print('http://api.census.gov/data/2010/sf1?key=1eb4e956ec8bdfd987960641728d0fed68589575&get=P0010001,NAME&for=state:*')
print(response.url)
print(response.text)

We can also write queries that obtain several data sets all at once. For example, we can obtain population by age and ethnicity using more codes. Codes can be found in the codebook

https://api.census.gov/data/2010/dec/sf1/variables.html

And we can restrict the query to a single state.

**Exercise** Pull the population totals for Black and White males that are 15 and 35 years old  for only Alaska

We can also retrieve the population for specific cities - pull the population totals those three categories for Chicago, Evanston, and Wilmette.

### Analysis

Which state has the largest African American population as a percentage of total population? Pull the data from the Census and also plot the percentage AA population per state in a graph.

## Social Platforms

Social platforms provide APIs that are of varying utility and continuing to constantly change in usefulness. Facebook's API has long been relatively useless, the Twitter API accessibility fundamentally changed after Elon Musk's acquisition, and even Reddit's API usage agreement was drastically changed in an effort to increase profitability for the company. Relying on the API access from private companies to data has proven to be a dangerous assumption in the long run. 

With that being said, these social platforms typically require some form of OAuth -- meaning that you have an account with the service and use those account credentials to log in. This will typically involve going to the site to create a custom application and using the `token` and `secret` that is provided for the application. Generally the set-up is well documented and easy enough to follow.